In [1]:
import requests
import sys
from os.path import join, expanduser
from pathlib import Path

import boto3
import fsspec
import numpy as np
import s3fs
import xarray as xr
import zarr
from dask.distributed import Client

user_home_dir = expanduser('~')
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

In [2]:
client = Client("tcp://127.0.0.1:42659")
client

<Client: 'tcp://127.0.0.1:42659' processes=4 threads=8, memory=61.46 GiB>

In [3]:
# Use this for the netcdf files stored on an s3 bucket
def get_credentials(use_earthdata: bool = False):
    """
    This routine automatically pulls your EDL crediential from .netrc file and use it to obtain an AWS S3 credential 
    through a PO.DAAC service accessible at https://archive.podaac.earthdata.nasa.gov/s3credentials.
    From the PO.DAAC Github (https://podaac.github.io/tutorials/external/July_2022_Earthdata_Webinar.html).
    
    Returns:
    =======
    
    credentials: a dictionary with AWS secret_key, access_key, and token
    """
    # NASA EarthData hosts ECCO V4r4 fileds
    if not use_earthdata:
        session = boto3.Session()
        credentials_b3 = session.get_credentials()        
        credentials = dict()
        credentials['secretAccessKey'] = credentials_b3.secret_key
        credentials['accessKeyId'] = credentials_b3.access_key
        credentials['sessionToken'] = credentials_b3.token

    # A 'public' AWS s3 bucket hosts V4r5 fields (they will eventually move to PO.DAAC)
    else:
        credentials = requests.get('https://archive.podaac.earthdata.nasa.gov/s3credentials').json()
    
    return credentials
    

def init_S3FileSystem(use_earthdata: bool = False, requester_pays: bool = True):
    """
    This routine automatically creates an 's3 file system' object and credentials dictionary.
    The s3 file system needs to be initialized with the special aws credentials.
    
    Returns:
    =======
    
    s3: an AWS S3 filesystem, 
    credentials: a dictionary with AWS secret_key, access_key, and token

    """
    credentials = get_credentials(use_earthdata=use_earthdata)

    if use_earthdata:
        requester_pays = False
        
    s3 = s3fs.S3FileSystem(
        requester_pays=requester_pays,
        anon=False,
        key=credentials['accessKeyId'],
        secret=credentials['secretAccessKey'], 
        token=credentials['sessionToken']
    )
    
    return s3, credentials


def list_s3_fsspec_ecco(mzz_dir: Path) -> None:
    print(np.sort(list(mzz_dir.glob("*.json"))))


def get_s3_fsspec_ecco(data_file: str, mzz_dir: Path, **kwargs) -> xr.Dataset:
    fs = fsspec.filesystem(
        "reference",     
        fo=str(mzz_dir / data_file),
        remote_protocol="s3",
        remote_options={"anon":False, "requester_pays":True},
        skip_instance_cache=True
    )
    fs.asynchronous = True
    store = zarr.storage.FsspecStore(fs)
    ds = xr.open_dataset(
        store,
        engine='zarr',
        consolidated=False,
        **kwargs,
    )
    return ds


def llc_to_latlon(
    xc,
    yc,
    field,
    min_lat: float = -90,
    max_lat: float = 90,
    dlat: float = 1,
    min_lon: float = -180,
    max_lon: float = 180,
    dlon: float = 1,
    method: str | None = None,
    radius: float | None = None,
):
    if method is None:
        method = "nearest_neighbor"
    if radius is None:
        radius = 120000
    lon_c, lat_c, _, _, field_latlon = ecco.resample_to_latlon(
        xc,
        yc, 
        field,
        min_lat,
        max_lat,
        dlat,
        min_lon,
        max_lon,
        dlon,
        fill_value = np.nan,
        mapping_method = method,
        radius_of_influence = radius,
    )
    if "k" in field.dims:
        dims = ["time", "lat", "lon", "depth"]
        coords = dict(
            time=field.time,
            lat=lat_c[:, 0],
            lon=lon_c[0],
            depth=field["Z"],
        )
    else:
        dims = ["time", "lat", "lon"]
        coords = dict(
            time=field.time,
            lat=lat_c[:, 0],
            lon=lon_c[0],
        )
    ds_latlon = xr.DataArray(
        data=field_latlon,
        dims=dims,
        coords=coords,
    )
    ds_latlon.attrs = field.attrs
    return ds_latlon

In [4]:
s3, credentials = init_S3FileSystem(use_earthdata=False, requester_pays=True)
mzz_dir = Path("/efs_ecco/mzz-jsons-V4r5/MZZ_day_mean_native")

In [5]:
ds_precip = get_s3_fsspec_ecco(
    "OCEAN_AND_ICE_SURFACE_FW_FLUX_day_mean_native_llc090_ECCOV4r5.json",
    mzz_dir,
    chunks={"time": 1, "tile": 13, "i": 90, "j": 90, "k": 50}
).sel(tile=4).isel(time=slice(100))
ds_ts = get_s3_fsspec_ecco(
    "OCEAN_TEMPERATURE_SALINITY_day_mean_native_llc090_ECCOV4r5.json",
    mzz_dir,
    chunks={"time": 1, "tile": 13, "i": 90, "j": 90, "k": 50}
).sel(tile=4).isel(k=0).isel(time=slice(100))

In [6]:
ds_ts

<xarray.Dataset> Size: 7MB
Dimensions:    (time: 100, j: 90, i: 90, nb: 4, j_g: 90, i_g: 90, nv: 2,
                k_l: 50, k_p1: 51, k_u: 50)
Coordinates: (12/22)
  * time       (time) datetime64[ns] 800B 1992-01-01T18:00:00 ... 1992-04-09T...
    XC         (j, i) float32 32kB dask.array<chunksize=(90, 90), meta=np.ndarray>
    XC_bnds    (j, i, nb) float32 130kB dask.array<chunksize=(90, 90, 4), meta=np.ndarray>
    XG         (j_g, i_g) float32 32kB dask.array<chunksize=(90, 90), meta=np.ndarray>
    YC         (j, i) float32 32kB dask.array<chunksize=(90, 90), meta=np.ndarray>
    YC_bnds    (j, i, nb) float32 130kB dask.array<chunksize=(90, 90, 4), meta=np.ndarray>
    ...         ...
    k          int32 4B 0
  * k_l        (k_l) int32 200B 0 1 2 3 4 5 6 7 8 ... 41 42 43 44 45 46 47 48 49
  * k_p1       (k_p1) int32 204B 0 1 2 3 4 5 6 7 8 ... 43 44 45 46 47 48 49 50
  * k_u        (k_u) int32 200B 0 1 2 3 4 5 6 7 8 ... 41 42 43 44 45 46 47 48 49
    tile       int32 4B 4
    time_bnds  (time, nv) datetime64[ns] 2kB dask.array<chunksize=(1, 2), meta=np.ndarray>
Dimensions without coordinates: nb, nv
Data variables:
    SALT       (time, j, i) float32 3MB dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    THETA      (time, j, i) float32 3MB dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
Attributes: (12/63)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty, Ou Wang, Ichiro Fukumori
    cdm_data_type:                   Grid
    comment:                         Fields provided on the curvilinear lat-l...
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    time_coverage_duration:          P1D
    time_coverage_end:               1992-01-02T00:00:00
    time_coverage_resolution:        P1D
    time_coverage_start:             1992-01-01T12:00:00
    title:                           ECCO Ocean Temperature and Salinity - Da...
    uuid:                            7601fbdc-2719-11f0-8ed8-0a58a9feac02

In [9]:
da_oceFWflx = llc_to_latlon(
    ds_precip["XC"],
    ds_precip["YC"],
    ds_precip["oceFWflx"],
    min_lat = -20,
    max_lat = 10,
    min_lon = 90,
    max_lon = 120,
)
da_oceFWflx.name = "surface_freshwater_flux"

In [7]:
da_theta = llc_to_latlon(
    ds_ts["XC"],
    ds_ts["YC"],
    ds_ts["THETA"],
    min_lat = -20,
    max_lat = 10,
    min_lon = 90,
    max_lon = 120,
)
da_theta.name = "sea_surface_temperature"

In [8]:
da_salt = llc_to_latlon(
    ds_ts["XC"],
    ds_ts["YC"],
    ds_ts["SALT"],
    min_lat = -20,
    max_lat = 10,
    min_lon = 90,
    max_lon = 120,
)
da_salt.name = "sea_surface_salinity"

In [9]:
ds = xr.merge([da_oceFWflx, da_theta, da_salt])

In [10]:
ds.to_netcdf("/efs_ecco/ascherer/datasets/surface_sal_temp_fw_daily_latlon_10N-20S_90E-120E.nc")